In [ ]:
#export
from local.imports import *
from local.test import *
from local.core import *
from local.layers import *
from local.data.pipeline import *
from local.data.source import *
from local.data.core import *
from local.data.external import *
from local.notebook.showdoc import show_doc
from local.optimizer import *
from local.learner import *

In [ ]:
#default_exp callback.progress

# Progress and logging callbacks

> Callback and helper function to track progress of training or log results

In [ ]:
from local.utils.test import *

## ProgressCallback -

In [ ]:
class ProgressCallback(Callback):
    order = 30 #needs to run after the Recorder
    def begin_fit(self):
        self.mbar = master_bar(list(range(self.n_epoch)))
        self.mbar.on_iter_begin()
        self.old_logger,self.learn.logger = self.logger,self._write_stats
        self._write_stats(self.recorder.metric_names)
    
    def begin_epoch(self):    self.mbar.update(self.epoch)
    def begin_train(self):    self._launch_pbar()
    def begin_validate(self): self._launch_pbar()
    def after_train(self):    self.pbar.on_iter_end()
    def after_validate(self): self.pbar.on_iter_end()
    def after_batch(self):    self.pbar.update(self.iter+1)
        
    def _launch_pbar(self):
        self.pbar = progress_bar(self.dl, parent=self.mbar)
        self.pbar.update(0)
        
    def after_fit(self):      
        self.mbar.on_iter_end()
        self.learn.logger = self.old_logger
    
    def _write_stats(self, log):
        self.mbar.write([f'{l:.6f}' if isinstance(l, float) else str(l) for l in log], table=True)

In [ ]:
learn = synth_learner(cbs=ProgressCallback())
learn.fit(10)

train_loss,valid_loss,time
10.237925,2.089526,00:00
0.732663,1.584957,00:00
1.680870,0.668579,00:00
0.171392,0.109632,00:00
0.141334,0.164688,00:00
0.055159,0.013948,00:00
0.018960,0.012242,00:00
0.016480,0.010544,00:00
0.010800,0.018214,00:00
0.010889,0.015344,00:00


In [ ]:
print(learn.progress.log)

begin epoch 0
begin train for 10 iterations
Iter 0
Iter 1
Iter 2
Iter 3
Iter 4
Iter 5
Iter 6
Iter 7
Iter 8
Iter 9
begin valid for 10 iterations
Iter 0
Iter 1
begin epoch 1
begin train for 10 iterations
Iter 0
Iter 1
Iter 2
Iter 3
Iter 4
Iter 5
Iter 6
Iter 7
Iter 8
Iter 9
begin valid for 10 iterations
Iter 0
Iter 1
begin epoch 2
begin train for 10 iterations
Iter 0
Iter 1
Iter 2
Iter 3
Iter 4
Iter 5
Iter 6
Iter 7
Iter 8
Iter 9
begin valid for 10 iterations
Iter 0
Iter 1
begin epoch 3
begin train for 10 iterations
Iter 0
Iter 1
Iter 2
Iter 3
Iter 4
Iter 5
Iter 6
Iter 7
Iter 8
Iter 9
begin valid for 10 iterations
Iter 0
Iter 1
begin epoch 4
begin train for 10 iterations
Iter 0
Iter 1
Iter 2
Iter 3
Iter 4
Iter 5
Iter 6
Iter 7
Iter 8
Iter 9
begin valid for 10 iterations
Iter 0
Iter 1
begin epoch 5
begin train for 10 iterations
Iter 0
Iter 1
Iter 2
Iter 3
Iter 4
Iter 5
Iter 6
Iter 7
Iter 8
Iter 9
begin valid for 10 iterations
Iter 0
Iter 1
begin epoch 6
begin train for 10 iterations
Iter 0
I